<a href="https://colab.research.google.com/github/nnn144/Research/blob/master/pseudo_heart_motion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import cv2 as cv
import numpy as np

from skimage.transform import radon
from numpy.random import choice

def crop_center(img, cropx, cropy):
    y, x = img.shape
    startx = x // 2 - (cropx // 2)
    starty = y // 2 - (cropy // 2)
    return img[starty:starty + cropy, startx:startx + cropx]
def Affine_Transformation_RTS_openCV(image, imp, ang=0, tx=0, ty=0, sc=0, sh=0):
    # get rows and columns of the image
    rows, cols = image.shape
####Rotation
    if imp == 'R':
        '''Function performs rotation using openCV, takes in ang as angle in degrees'''
        M = cv.getRotationMatrix2D(((cols-1)/2.0,(rows-1)/2.0),-ang,1)
        out = cv.warpAffine(image,M,(cols,rows))#bilinear interpolation
        return out

######Scaling
    if imp == 'S':
        '''Function performs scaling using openCV, takes in scaling factor, crops image to original size'''
        new_width = math.floor(sc*cols)
        new_height = math.floor(sc*rows)
        scaled = cv.resize(image,(new_width, new_height), interpolation = cv.INTER_LINEAR) #bilinear interpolation
        out = crop_center(scaled, cols, rows)
        return out


#######Translation
    if imp == 'T':
        '''Function performs translation using openCV, takes in x and y values'''
        M = np.float32([[1,0,tx],[0,1,ty]])
        out = cv.warpAffine(image,M,(cols,rows))
        return out

#######Translation
    if imp == "SH":
        # get shear matrix
        shear_scale_x = shear_scale_y = sh
        shear_mat = np.array([[1, shear_scale_x, 0],
                              [shear_scale_y, 1, 0]]).astype("float32")
        # compute the shearing image
        shear_img = cv.warpAffine(image, shear_mat, (rows,cols))

        return shear_img


In [2]:
class Heart_motion:
    def __init__(self, im_t, ang, sc, xt, yt, m_rate, m_time):
        self.im_t = im_t  #number of frames
        self.ang = ang  #angle (degrees) (1-->1.5, floats) (-1 --> -1.5)
        self.sc = sc  #scaling (scales up) :1.05 (.01 --> .06)
        self.xt = xt  #x translation  (1-->4, floats) (-1 --> -4 floats)
        self.yt = yt  #y translation  (1-->4, floats) (-1 --> -4 floats)
        self.m_rate = m_rate  #motion rate for R S T
        self.m_time = m_time  #motion time R S T

    def generate_motion(self, data_for_transformation):
        #One direction movement
        transforms = 'RST'
        ran = len(transforms)
        for t in range(0,ran):
            #Rotation, Scaling, Translation
            #print(transforms[t])
            #angle, translation x, translation y, scaling factor
            ang_count = 0  #angle
            trx_count = 0  #x direction
            try_count = 0  #y direction
            sc_count = 0   #scaling
            temp = []
            for i in range(0,self.im_t):
                #print(trx_count)
                if i <= self.m_time[t]:
                    temp_img = Affine_Transformation_RTS_openCV(data_for_transformation[i], transforms[t],
                                                                self.ang*ang_count, self.xt*trx_count,
                                                                self.yt*try_count, 1+(self.sc*sc_count))
                    data_for_transformation[i] = temp_img
                    temp.append(temp_img)
                    ang_count+=1
                    trx_count+=1
                    try_count+=1
                    sc_count+=1

                elif i > self.m_time[t]:
                    temp_id = i % self.m_rate[t]
                    #print (temp_id, len(temp))
                    if temp_id < len(temp):
                        data_for_transformation[i] = temp[temp_id]
                        ang_count+=1
                        trx_count+=1
                        try_count+=1
                        sc_count+=1
                    else:
                        data_for_transformation[i] = data_for_transformation[i]
                        ang_count = 0
                        trx_count = 0
                        try_count = 0
                        sc_count = 0
        # done. Return the array
        return np.asarray(data_for_transformation)


In [ ]:
# base images for motions
path = "../../../../store01/groups/scw1489/Haoran/valerie_data/"
aug_noisy_list = np.load(path+"noisy_list.npy")
x_train = []
y_train = []
for ground_truth in aug_noisy_list:
    """
    Randomly create parameters for motion
    Roughly two types:
    1. high sampling rate -> small move for every frame
    2. low sampling rate -> large move for every frame
    """
    # HIGH sampling rate
    im_t = 100  #number of frames
    data_for_transformation = [] #run every new mouse
    for x in range(im_t):
        data_for_transformation.append(ground_truth)

    # rotation angle range: angle (degrees) (1-->1.5, floats) (-1 --> -1.5)
    ang_range = np.concatenate((np.arange(10, 16), -np.arange(10, 16)))/10
    ang = choice(ang_range, size=1)
    #scaling (scales up) :1.05 (.01 --> .06)
    sc = choice(np.arange(101, 107)/100, size=1)
    #translation  (1-->4, floats) (-1 --> -4 floats)
    tr_range = np.concatenate((np.arange(10, 50), -np.arange(10, 50)))/10
    xt, yt = choice(tr_range, 2)
    m_rate = [7, 7, 7]  #motion rate for R S T
    m_time = [4, 4, 4]  #motion time R S T
    # create motions
    mheart1 = Heart_motion(im_t, ang, sc, xt, yt, m_rate, m_time)
    motion_imgs = mheart1.generate_motion(data_for_transformation)

    # HIGH sampling m_rate
    temp = motion_imgs[0]
    count = 0
    for i in np.arange(1, 100, step=2):
        temp += motion_imgs[i]
        count += 1
    x_train.append(temp/count)
    y_train.append(ground_truth)
    
    # ====================================================================
    # LOW sampling m_rate
    # rotation angle range: angle (degrees) (1-->1.5, floats) (-1 --> -1.5)
    ang = ang/10
    #scaling (scales up) :1.05 (.01 --> .06)
    sc = sc/10
    #translation  (1-->4, floats) (-1 --> -4 floats)
    xt, yt = xt/10, yt/10
    m_rate = [70, 70, 70]  #motion rate for R S T
    m_time = [40, 40, 40]  #motion time R S T
    # create motions
    mheart2 = Heart_motion(im_t, ang, sc, xt, yt, m_rate, m_time)
    motion_imgs = mheart2.generate_motion(data_for_transformation)
    temp = motion_imgs[0]
    count = 0
    for i in np.arange(1, 50):
        temp += motion_imgs[i]
        count += 1
    x_train.append(temp/count)
    y_train.append(ground_truth)

# compute the avg image and get the sinogram
theta = np.linspace(0., 180., 64, endpoint=True)
fbp_test = []
for k, img in enumerate(x_train):
    # compute the sinogram of average
    sino = radon(temp, theta=theta)
    #fbp_test.append(radon(temp, theta=theta, circle=False))
    x_train[k] = sino

np.save("y_train2", y_train)
np.save("x_train2", x_train)
#np.save("y_test1", base_test)
